# Content

Due to the complexity associated to web scraping, it is usual to get data with small imperfections which are easy to correct. In this notebook we are going to see how the data is automatically cleaned. The whole process is summarised in the preprocessing.py script. 

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import ast

In [26]:
data = pd.read_csv('../data/raw/amazon_shop_data_boxing_fighting_gloves.csv')

In [27]:
data.head()

,title,brand,rating,ratings_number,price,prime_option,answers,first_date,sponsored_option,categories,position,page
0,RDX Boxing Gloves Sparring and Muay Thai Maya ...,Visit the RDX Store,4.9 out of 5 stars,93 ratings,28.0,True,0,\n6 Nov. 2020\n,True,['882 in Sports & Outdoors (See Top 100 in Spo...,0,1
1,XN8 Boxing Gloves for Training Punch bag - MMA...,Visit the Xn8 Sports Store,4.6 out of 5 stars,306 ratings,23.0,True,\n5 answered questions\n,\n21 July 2020\n,True,"['3,900 in Sports & Outdoors (See Top 100 in S...",1,1
2,"RDX MMA Gloves Grappling Sparring, Pre-Curved ...",Visit the RDX Store,4.9 out of 5 stars,13 ratings,22.0,True,0,\n13 Nov. 2020\n,True,"['4,381 in Sports & Outdoors (See Top 100 in S...",2,1
3,RDX Boxing Gloves for Training Muay Thai Maya ...,Visit the RDX Store,4.7 out of 5 stars,"1,631 ratings",31.0,True,\n28 answered questions\n,\n5 Oct. 2018\n,True,"['2,252 in Sports & Outdoors (See Top 100 in S...",3,1
4,EVO Boxing Gloves with Hand Wraps For Men and ...,Brand: EVO Fitness,4.6 out of 5 stars,619 ratings,21.0,True,\n12 answered questions\n,\n6 Feb. 2014\n,False,['570 in Sports & Outdoors (See Top 100 in Spo...,4,1


In [28]:
data.shape

(360, 12)

In [29]:
data.isnull().sum()

title                0
brand                0
rating              68
ratings_number      68
price                6
prime_option         0
answers              0
first_date           7
sponsored_option     0
categories           0
position             0
page                 0
dtype: int64

# Brand

In [30]:
data['brand'].head(5)

0           Visit the RDX Store
1    Visit the Xn8 Sports Store
2           Visit the RDX Store
3           Visit the RDX Store
4            Brand: EVO Fitness
Name: brand, dtype: object

We can see that the brand is included in two different phrases: 

1. 'Brand: {brand}'
2. Visit the {brand} Store

So, to be simple, we just have to take the proper indexes of the string values depending if 'Visit' or 'Brand' is a substring.

In [31]:
def get_brand(x):
    
    if 'Visit' in x:
        return x[10:][:-6]
    elif 'Brand' in x:
        return x[7:]
    else:
        return x
    
data['brand'] = np.vectorize(get_brand)(data['brand'])

# Rating

In [32]:
data['rating'].head()

0    4.9 out of 5 stars
1    4.6 out of 5 stars
2    4.9 out of 5 stars
3    4.7 out of 5 stars
4    4.6 out of 5 stars
Name: rating, dtype: object

As we can see, the rating has been collected as the string '{rating} out of 5 stars'. So, we just have to take the first three components.

In [33]:
data['rating'] = data['rating'].apply(lambda x: x[:3] if type(x)==str else x).astype(float)

# Ratings number

In [34]:
data['ratings_number'].head()

0       93 ratings
1      306 ratings
2       13 ratings
3    1,631 ratings
4      619 ratings
Name: ratings_number, dtype: object

The ratings number has been collected as the string '{ratings_number} ratings', so, we just have to split the string by whitespaces and take the first element.

In [35]:
data['ratings_number'] = data['ratings_number'].apply(lambda x:
                                                          int(x.split(' ')[0].replace(',', '')) if type(x)==str else 0)

# Answers

In [36]:
data['answers']

0                              0
1       \n5 answered questions\n
2                              0
3      \n28 answered questions\n
4      \n12 answered questions\n
                 ...            
355     \n6 answered questions\n
356                            0
357                            0
358                            0
359     \n6 answered questions\n
Name: answers, Length: 360, dtype: object

As we can see, we have two different values, a string '\n{ratings_number} answered questions' and a '0'. In the first case we just have to split the string by the whitespaces and take the last characters starting from the second character of the first split. In the second case we will transform the value into integer type.

In [37]:
data['answers']=data['answers'].apply(lambda x:
                                          int(x[1:-2].split(' ')[0]) if '\n' in x else x).astype(int)

# First date

In [38]:
data['first_date']

0        \n6 Nov. 2020\n
1       \n21 July 2020\n
2       \n13 Nov. 2020\n
3        \n5 Oct. 2018\n
4        \n6 Feb. 2014\n
             ...        
355     \n3 April 2015\n
356      \n9 Oct. 2020\n
357    \n13 April 2020\n
358     \n13 July 2020\n
359      \n6 Feb. 2014\n
Name: first_date, Length: 360, dtype: object

Amazon uses the format 'day month year' (ex: 23 Jan. 2019), where the the month is not standarised. They use common names but July, April and September are not abbreviated properly. In addition, they use points and Python datetimes does not. 

In order to standarise the values we transform the values with the next function.

In [39]:
def get_date(date):  # string dates
    
    if  type(date)!= str:
        return np.nan
    
    new_date = date[1:-1].replace('.', '')
    different_months = {'July': 'Jul', 'April': 'Apr', 'Sept': 'Sep'}
    date_elements = new_date.split(' ')
    
    # checking integrity of the date
    
    if len(date_elements) != 3:
        return np.nan
    
    if date_elements[1] not in list(different_months.keys()):
        result = datetime.datetime.strptime(new_date, '%d %b %Y')
        
    else:
        date_elements[1] = different_months[date_elements[1]]
        processed_date = date_elements[0]+' '+date_elements[1]+' '+date_elements[2]
        
        result = datetime.datetime.strptime(processed_date, '%d %b %Y')
    
    return result

In [40]:
data['first_date'] = data['first_date'].apply(lambda x: get_date(x))

# Category and position

In [41]:
data['categories'].loc[0]

"['882 in Sports & Outdoors (See Top 100 in Sports & Outdoors)', '3 in Boxing Training Gloves']"

In this case we notice that, usually, we have two values in the list: A general category and a particular category inside the general. In both cases we are just interested in the position and the name of the categories:

1. General category: '{position} in {category} (See Top 100 in Sports & Outdoors)'
2. Specific category: '{position} in {specific category}'

In both cases we will remove the phrase between the parenthesis, remove special characters (such as # or &) and split by whitespaces in order to remove the connector 'in'.

In [42]:
def get_categories_and_positions(strings_array):  # ---> resolver
    
    strings_array = ast.literal_eval(strings_array) # evaluate the list
    result = []
    for category in strings_array:
        
        position, name = None, ''
        
        if 'Amazon Bestsellers Rank:' in category:
            for char in ['#', ',']:
                category = category.replace(char, '')
            
            position = int(category.split(' ')[category.split(' ').index('Rank:')+1].replace(',', ''))
            
            if '(' in category:
                init_ind = category.index('(')
                category = category[:init_ind]
            
            for x in category.split(' ')[5:]:
                name = name+' '+x
            
            result.append(name.strip()), result.append(position)
            continue
            
        chars=[]
        for char in ['#', ',']:
            chars = category.replace(char, '')
        
        if '(' in chars:
            init_ind = chars.index('(')
            chars = chars[:init_ind]
        
        chars = chars.strip().split(' ')
        position = int(chars[0].replace('#', ''))

        for x in range(2,len(chars)):
            name = name  + ' ' + chars[x]
            
        result.append(name.strip()), result.append(position)
        
    return result

In [43]:
categories_positions = pd.DataFrame(
    data['categories'].apply(lambda x: get_categories_and_positions(x)).tolist()
).loc[:, 0:3]

categories_positions.columns = ['category', 'position_c', 'subcategory', 'position_s']

data = pd.concat([data, categories_positions], axis=1)

In [44]:
data.isnull().sum()

title                0
brand                0
rating              68
ratings_number       0
price                6
prime_option         0
answers              0
first_date          22
sponsored_option     0
categories           0
position             0
page                 0
category            21
position_c          21
subcategory         21
position_s          21
dtype: int64

# Export to csv and finish

In [45]:
data.to_csv('../data/interim/amazon_shop_data_boxing_fighting_gloves_interim.csv', index=False)